# MNSIT-Sequential模板

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import sampler
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.datasets as dset
import torchvision.transforms as T
import numpy as np
import timeit
%matplotlib inline

# 载入数据

In [2]:
set_train = dset.MNIST('./MNIST', train=True, transform=T.ToTensor(), download=True)
loader_train = DataLoader(set_train, batch_size=64)
set_test = dset.MNIST('./MNIST', train=False, transform=T.ToTensor(),download=True)
loader_test = DataLoader(set_test, batch_size=64)

print("训练集大小：",set_train.train_data.size())
print("训练集标签：",set_train.train_labels.size())
print("测试集大小：",set_test.test_data.size())
print("测试集标签：",set_test.test_labels.size())
type(set_train[0]) #数据集的索引是tuple

训练集大小： torch.Size([60000, 28, 28])
训练集标签： torch.Size([60000])
测试集大小： torch.Size([10000, 28, 28])
测试集标签： torch.Size([10000])


tuple

# 初始化一些参数

In [3]:
# 打印loss的频率
print_every = 100

# This is a little utility that we'll use to reset the model
# if we want to re-initialize all our parameters
def reset(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()
        
class Flatten(nn.Module):
    def forward(self, x):
        N, C, H, W = x.size() # 读取 N, C, H, W
        return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image
    
def train(model, loss_fn, optimizer, num_epochs = 1):
    for epoch in range(num_epochs):
        print('Starting epoch %d / %d' % (epoch + 1, num_epochs))
        check_accuracy(Net, loader_test)
        model.train()
        for t, (x, y) in enumerate(loader_train):
            x_var = Variable(x.cuda())
            y_var = Variable(y.cuda().long())
            scores = model(x_var)
            loss = loss_fn(scores, y_var)
            if (t + 1) % print_every == 0:
                print('t = %d, loss = %.4f' % (t + 1, loss.data[0]))
                   
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

def check_accuracy(model, loader):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    for x, y in loader:
        x_var = Variable(x.cuda(), volatile=True)

        scores = model(x_var)
        _, preds = scores.data.cpu().max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

# 构造模型S

In [4]:
Net = nn.Sequential( 
                    nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1), #conv1
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(kernel_size=2,stride=2), #pool1
                    nn.BatchNorm2d(16), #bn1
                    nn.Conv2d(16, 32, kernel_size=3, stride=1,padding=0), #conv2
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(kernel_size=2,stride=2), #pool2
                    nn.BatchNorm2d(32), #bn2
                    Flatten(), 
                    nn.Linear(1152, 400),                    
                    nn.ReLU(inplace=True),
                    nn.Linear(400, 200),                    
                    nn.ReLU(inplace=True),
                    nn.Linear(200, 100),                    
                    nn.ReLU(inplace=True),
                    nn.Linear(100, 10),                   
            )

net =Net.cuda() #实例化模型

x = torch.randn(64, 1, 28, 28) #测试模型是否架构数据
x_var = Variable(x).cuda() # 变量化
ans = net(x_var)        # 喂入模型

# 应该相等
print(ans.size())
np.array_equal(np.array(ans.size()), np.array([64, 10]))

torch.Size([64, 10])


True

In [5]:
net2 = torch.load("net_s.pkl")

In [6]:
net2

Sequential (
  (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU (inplace)
  (2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True)
  (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (5): ReLU (inplace)
  (6): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
  (8): Flatten (
  )
  (9): Linear (1152 -> 400)
  (10): ReLU (inplace)
  (11): Linear (400 -> 200)
  (12): ReLU (inplace)
  (13): Linear (200 -> 100)
  (14): ReLU (inplace)
  (15): Linear (100 -> 10)
)

In [7]:
net2 = torch.load("alexnet.pkl")

In [8]:
net2

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 1000)
  )
)

In [14]:
x = torch.rand(1,32,32,3)
x = Variable(x)

In [15]:
net2(x)

RuntimeError: Given input size: (3 x 32 x 3). Calculated output size: (64 x 7 x 0). Output size is too small at d:\downloads\pytorch-master-1\torch\lib\thnn\generic/SpatialConvolutionMM.c:45